In [1]:
from nltk.corpus import wordnet as wn
from nltk import pos_tag, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def disambiguate_nouns(word1, word2):
    # Obtener los synsets (conjunto de sinónimos) para cada palabra
    synsets_word1 = wn.synsets(word1, pos=wn.NOUN)
    synsets_word2 = wn.synsets(word2, pos=wn.NOUN)
    # Calcular la similitud de coseno entre los vectores de frecuencia de las definiciones de los synsets
    vectorizer = CountVectorizer()
    synset_definitions = [synset.definition() + ' ' + ' '.join([word.name() for word in synset.hypernym_paths()[0]])  for synset in synsets_word1]
    synset_definitions = synset_definitions + [synset.definition() + ' ' + ' '.join([word.name() for word in synset.hypernym_paths()[0]])  for synset in synsets_word2]
    vector_space = vectorizer.fit_transform(synset_definitions)
    similarity_matrix = cosine_similarity(vector_space)
    # Encontrar la definición más similar para cada palabra
    max_value = -1
    row_index = -1
    col_index = -1
    for row in range(len(synsets_word1)):
        for col in range(len(synsets_word1), len(synset_definitions)):
            if col != row:
                if similarity_matrix[row,col] > max_value:
                    max_value = similarity_matrix[row,col]
                    row_index = row
                    col_index = col

    # Devolver el synset con la definición más similar para cada palabra
    best_synset_word1 = synsets_word1[row_index]
    best_synset_word2 = synsets_word2[col_index-len(synsets_word1)]

    return best_synset_word1, best_synset_word2

# Ejemplo de uso
word1 = "cat"
word2 = "dog"
best_synset_word1, best_synset_word2 = disambiguate_nouns(word1, word2)
print(best_synset_word1.definition())
print(best_synset_word2.definition())


any of several large cats typically able to roar and living in the wild
a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds
